We will use "IMDB movie review sentiment classification dataset"

Dataset Description: https://keras.io/api/datasets/imdb/

This is a dataset of 25,000 movie reviews from IMDB, tagged by sentiment (positive/negative). The reviews have been preprocessed and each review is coded as a list of (whole) word indexes. For convenience, words are indexed by their overall frequency in the dataset, so that, for example, the integer "3" encodes the 3rd most frequent word in the data.

In [1]:
#!pip install keras

In [2]:
#!pip install tensorflow

In [3]:
#! pip install tensorflow --upgrade

In [25]:
import numpy
import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Dropout
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers.convolutional import Conv1D
from tensorflow.python.keras.layers.convolutional import MaxPooling1D
from keras.preprocessing import sequence
#from tensorflow.python.keras.utils import one_hot
from keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.layers import Flatten
# fix random seed for reproducibility;pl
numpy.random.seed(7)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
db=imdb.load_data()

17464789/17464789 [==============================] - 0s 0us/step


In [6]:
#db

In [5]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [7]:
X_train.shape

(25000,)

In [8]:
X_train

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]),
       list([1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 4369,

In [9]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [11]:
#from keras.preprocessing import sequence
from keras.utils import pad_sequences
max_review_length = 500
X_train = pad_sequences(X_train, maxlen=max_review_length)
X_test = pad_sequences(X_test, maxlen=max_review_length)

In [13]:
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [14]:
X_train.shape

(25000, 500)

we will use the embedding layer which defines the first hidden layer of the network. it must specify 3 arguments:

input_dim: the size of the vocabulary in the text

output_dim: this is the size of the vector space in which each word will be immersed

input_legth: this is the size of the sequence, for example if your documents contain 100 words each then it is 100

In [15]:
# creating tyhe model 
embedding_vecor_length = 32

model = Sequential()

model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))

model.add(MaxPooling1D(pool_size=2))

model.add(LSTM(100))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.build((None, max_review_length))  # Build the model

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 module_wrapper (ModuleWrapp  (None, 500, 32)          160000    
 er)                                                             
                                                                 
 module_wrapper_1 (ModuleWra  (None, 500, 32)          3104      
 pper)                                                           
                                                                 
 module_wrapper_2 (ModuleWra  (None, 250, 32)          0         
 pper)                                                           
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                        

In [16]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Epoch 1/3
391/391 [==============================] - 74s 159ms/step - loss: 0.4132 - accuracy: 0.7975 - val_loss: 0.3058 - val_accuracy: 0.8764
Epoch 2/3
391/391 [==============================] - 32s 83ms/step - loss: 0.2416 - accuracy: 0.9060 - val_loss: 0.2796 - val_accuracy: 0.8878
Epoch 3/3
391/391 [==============================] - 18s 47ms/step - loss: 0.1946 - accuracy: 0.9257 - val_loss: 0.2810 - val_accuracy: 0.8832


In [17]:
# evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 88.32%


## a simple example of the embedding layer

In [18]:
docs = ['Well done!','Good work','Great effort','nice work','Excellent!',
'Weak','Poor effort!','not good','poor work','Could have done better.']

In [21]:
labels = [1,1,1,1,1,0,0,0,0,0]

In [22]:
vocab_size = 50

In [23]:
encoded_docs = [one_hot(d, vocab_size) for d in docs]

In [24]:
print(encoded_docs)

[[40, 44], [5, 20], [42, 31], [22, 20], [31], [29], [7, 31], [16, 5], [7, 20], [28, 5, 44, 5]]


In [25]:
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[40 44  0  0]
 [ 5 20  0  0]
 [42 31  0  0]
 [22 20  0  0]
 [31  0  0  0]
 [29  0  0  0]
 [ 7 31  0  0]
 [16  5  0  0]
 [ 7 20  0  0]
 [28  5 44  5]]


We are now ready to define our Embedding layer as part of our model.

The embedding has a vocabulary of 50 and an entry length of 4. We will choose a small embedding space of 8 dimensions.

The model is a simple binary classification model. It is important to note that the output of the Embedding layer will be 4 vectors of 8 dimensions each, one for each word. We flatten it (the flatten layer) into a 32-element vector to pass it to the Dense output layer. 

In [26]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# summarize the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              400       
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________


In [27]:
import numpy as np
padded_docs = np.array(padded_docs)
labels = np.array(labels)

In [28]:
print(padded_docs.shape)
print(labels.shape)

(10, 4)
(10,)


In [29]:
# Now you can proceed with model training
model.fit(padded_docs, labels, epochs=10, verbose=0)

In [30]:
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 69.999999


## To Do: 

1. Try the same thing on Google reviews dataset ( the file is given in the lab directory)
2. try to change the embedding representation using Glove and Skipgram 

In [163]:
# 1 Try the same thing on Google reviews dataset

In [26]:
import pandas as pd
df = pd.read_csv('/kaggle/input/data-reviews/reviews.csv')

In [27]:
df.head(2)

,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,Andrew Thomas,https://lh3.googleusercontent.com/a-/AOh14GiHd...,Update: After getting a response from the deve...,1,21,4.17.0.3,2020-04-05 22:25:57,"According to our TOS, and the term you have ag...",2020-04-05 15:10:24,most_relevant,com.anydo
1,Craig Haines,https://lh3.googleusercontent.com/-hoe0kwSJgPQ...,Used it for a fair amount of time without any ...,1,11,4.17.0.3,2020-04-04 13:40:01,It sounds like you logged in with a different ...,2020-04-05 15:11:35,most_relevant,com.anydo


In [28]:
df.columns

Index(['userName', 'userImage', 'content', 'score', 'thumbsUpCount',
       'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'sortOrder',
       'appId'],
      dtype='object')

In [29]:
X = df.content
Y = df.score

In [30]:
labels = []
for value in Y.values:
    if value <= 2:
        labels.append(0)
    else :
        labels.append(1)
print(len(labels))

15746


In [31]:
X.values[0]

"Update: After getting a response from the developer I would change my rating to 0 stars if possible. These guys hide behind confusing and opaque terms and refuse to budge at all. I'm so annoyed that my money has been lost to them! Really terrible customer experience. Original: Be very careful when signing up for a free trial of this app. If you happen to go over they automatically charge you for a full years subscription and refuse to refund. Terrible customer experience and the app is just OK."

In [33]:
import numpy as np
import string

text_list = X.tolist()
text = " ".join(text_list)
text = text.translate(str.maketrans('', '', string.punctuation))

text = text.lower()


words = text.split()

unique_words = set(words)

# Taille du vocabulaire
vocab_size = len(unique_words)
print("Taille du vocabulaire du content:", vocab_size)

Taille du vocabulaire du content: 12974


In [34]:
encoded_docs = [one_hot(text, vocab_size) for text in X]

In [35]:
len(encoded_docs)

15746

In [36]:
max_l = 0
for i in range(len(encoded_docs)):
    taille = len(encoded_docs[i])
    if taille > max_l:
        max_l = taille

max_l

393

In [37]:
padded_docs = pad_sequences(encoded_docs, maxlen = max_l, padding='post')
print(padded_docs)

[[10196  1912  2923 ...     0     0     0]
 [10834 12853  5634 ...     0     0     0]
 [ 3775  5373  5307 ...     0     0     0]
 ...
 [12055   263  5634 ...     0     0     0]
 [ 7458 12853     0 ...     0     0     0]
 [ 7346  3738  6303 ...     0     0     0]]


In [38]:
# me
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(vocab_size, 150, input_length=max_l))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 393, 150)          1946100   
                                                                 
 flatten (Flatten)           (None, 58950)             0         
                                                                 
 dense (Dense)               (None, 1)                 58951     
                                                                 
Total params: 2,005,051
Trainable params: 2,005,051
Non-trainable params: 0
_________________________________________________________________


In [40]:
padded_docs = np.array(padded_docs)
labels = np.array(labels)
print(labels.shape)
print(padded_docs.shape)

(15746,)
(15746, 393)


In [41]:
# Now you can proceed with model training
model.fit(padded_docs, labels, epochs=2, verbose=0)

In [43]:
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 97.288197


In [84]:
# 2- try to change the embedding representation using Glove and Skipgram

In [44]:
import nltk
import emoji

In [45]:
X = df.content.values
Y = labels

In [46]:
X[0:4]

array(["Update: After getting a response from the developer I would change my rating to 0 stars if possible. These guys hide behind confusing and opaque terms and refuse to budge at all. I'm so annoyed that my money has been lost to them! Really terrible customer experience. Original: Be very careful when signing up for a free trial of this app. If you happen to go over they automatically charge you for a full years subscription and refuse to refund. Terrible customer experience and the app is just OK.",
       'Used it for a fair amount of time without any problems. Suddenly then asked me to create an account or log using Google or FB. I used my Google one only to discover everything was gone!',
       "Your app sucks now!!!!! Used to be good but now doesn't update until I physically open it up and then close it then scroll and then it finally shows what I want to put on the list!!!!!! And I FRIGGEN paid for this garbage!!!!!!!",
       "It seems OK, but very basic. Recurring tasks ne

In [47]:
Y

array([0, 0, 0, ..., 1, 1, 1])

In [51]:
all_texts_p_t = [nltk.word_tokenize(emoji.demojize(t.lower(), language='en')) for t in X]
all_texts_preprocessed = [' '.join(text) for text in all_texts_p_t]

In [52]:
all_texts_preprocessed[15:17]

['this was originally a 5 star app , but now you can only use it by allowing it access to either your emails or facebook ! : enraged_face : :enraged_face : :enraged_face : i am definitely not happy to allow this so have deleted the app . would give it zero stars if i could .',
 "could n't even set up one single recurring reminder without being required to pay monthly subscription . get your greedy hand off it dev . it 's only only a reminder app not a useful service ."]

In [53]:
X = all_texts_preprocessed

In [55]:
from keras.preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(X)
vocab_size = len(t.word_index) + 1
#print(vocab_size)
# integer encode the documents
encoded_docs = t.texts_to_sequences(X)
max_l = 0
for i in range(len(encoded_docs)):
    taille = len(encoded_docs[i])
    if taille > max_l:
        max_l = taille

print(max_l)
#print(encoded_docs)
padded_docs = pad_sequences(encoded_docs, maxlen=max_l, padding='post')
print(padded_docs)

399
[[ 103   79  228 ...    0    0    0]
 [ 122    4    9 ...    0    0    0]
 [  59    7 1037 ...    0    0    0]
 ...
 [ 157  126    9 ...    0    0    0]
 [  55    4    0 ...    0    0    0]
 [  51  262    5 ...    0    0    0]]


In [56]:
from numpy import asarray
embeddings_index = dict()
f = open('/kaggle/input/glove6b/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [59]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [62]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(vocab_size, 100,weights=[embedding_matrix] ,input_length= max_l))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# summarize the model
model.summary()

model.fit(padded_docs, labels, epochs=5, batch_size=32)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 399, 100)          1157400   
                                                                 
 flatten_2 (Flatten)         (None, 39900)             0         
                                                                 
 dense_2 (Dense)             (None, 1)                 39901     
                                                                 
Total params: 1,197,301
Trainable params: 1,197,301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
493/493 [==============================] - 32s 63ms/step - loss: 0.5112 - accuracy: 0.7443
Epoch 2/5
493/493 [==============================] - 10s 20ms/step - loss: 0.3166 - accuracy: 0.8654
Epoch 3/5
493/493 [==============================] - 7s 13ms/step - loss: 0.2132 - accuracy: 0.9246
Epoch 

In [64]:
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 98.202717
